In [1]:
import functools
import os

import jax
from jax.experimental import pallas as pl
from jax.experimental.pallas import tpu as pltpu
import jax.numpy as jnp
from jax.sharding import NamedSharding, PartitionSpec as P

import numpy as np

In [2]:
jax.config.update('jax_num_cpu_devices', 4)

In [3]:
from pathlib import Path
import subprocess

def project_root() -> Path:
    return Path(subprocess.check_output(
        ['git', 'rev-parse', '--show-toplevel']
    ).decode().strip())

TRACES_DIR = project_root() / "traces"

In [4]:
m, k, n = 2048, 2048, 1024

k1, k2 = jax.random.split(jax.random.key(0), 2)
inputs = jax.random.normal(k1, (m, k), dtype=jnp.bfloat16)
weights = jax.random.normal(k2, (k, n), dtype=jnp.bfloat16)

In [5]:
mesh = jax.make_mesh((2, 2), ("x", "y"))
inp_sharding = jax.NamedSharding(mesh, P('x', 'y'))
w_sharding = jax.NamedSharding(mesh, P('x', None))

inputs = jax.device_put(inputs, inp_sharding)
weights = jax.device_put(weights, w_sharding)

/var/folders/my/845wytg53ln_99j7k_dzpl840000gn/T/ipykernel_74221/3396536806.py:1: DeprecationWarning: The default axis_types will change in JAX v0.9.0 to jax.sharding.AxisType.Explicit. To maintain the old behavior, pass `axis_types=(jax.sharding.AxisType.Auto,) * len(axis_names)`. To opt-into the new behavior, pass `axis_types=(jax.sharding.AxisType.Explicit,) * len(axis_names)
  mesh = jax.make_mesh((2, 2), ("x", "y"))


inputs are size 2048, 2048 -> bf16:: 2 bytes * 2048 * 2048 = ~8MB
weights are size 2048, 1024 -> bf16:: 2 bytes * 2048 * 1024 = ~4MB

inputs sharded along x and y -> $Inp[I_{X}, J_{Y}]$

weights sharded along x -> $W[J_{X}, K]$

Each device has N elements per array:
  - inputs
    - (2048 / 2) * (2048 / 2) * 2bytes
    - ~2MB
  - weights
    - (2048 / 2) * 1024 * 2bytes
    - ~2MB

The contracting dimension is sharded in both inputs and weights, along different axes.
Need to handle that with collectives; AG/AR

In [6]:
jax.debug.visualize_array_sharding(inputs)

                        
                        
   CPU 0       CPU 1    
                        
                        
                        
                        
                        
   CPU 2       CPU 3    
                        
                        
                        

In [7]:
jax.debug.visualize_array_sharding(weights)

            
            
  CPU 0,1   
            
            
            
            
            
  CPU 2,3   
            
            
            

In [8]:
def basic_matmul(x: jax.Array, y: jax.Array) -> jax.Array:
    return jnp.matmul(x, y)

out = basic_matmul(inputs, weights)
compiled = jax.jit(basic_matmul)

In [9]:
jax.debug.visualize_array_sharding(out)

            
            
  CPU 0,1   
            
            
            
            
            
  CPU 2,3   
            
            
            

In [10]:
result = compiled(inputs, weights)
result.block_until_ready()

with jax.profiler.trace(TRACES_DIR):
    result = compiled(inputs, weights)
    result.block_until_ready()

In [10]:
@functools.partial(
    jax.shard_map,
    mesh=mesh,
    in_specs=(P('x', 'y'), P('x', None)),
    out_specs=P('x', None),
    check_vma=False
)
def xla_matmul(input_shard: jax.Array, w_shard: jax.Array) -> jax.Array:
    # First we want to all_gather the data
    with jax.named_scope('all_gather(s)'):
        input_full = jax.lax.all_gather(input_shard, 'y', axis=1, tiled=True)
        w_full = jax.lax.all_gather(w_shard, 'x', axis=0, tiled=True) # gather w along x
    # Then we want to compute on the data
    with jax.named_scope('dot'):
        local_out = input_full @ w_full
    # Then we want to all reduce the data
    # with jax.named_scope('all_reduce'):
    #     out = jax.lax.psum(local_out, 'y')
    return local_out

In [11]:
# https://docs.jax.dev/en/latest/notebooks/shard_map.html
from jax.tree_util import tree_map, tree_all

def allclose(a, b):
  return tree_all(tree_map(functools.partial(jnp.allclose, atol=1e-2, rtol=1e-2), a, b))

allclose(xla_matmul(inputs, weights), jnp.dot(inputs, weights))

True

In [13]:
gemm2_compiled = jax.jit(xla_matmul)
result = gemm2_compiled(inputs, weights)
result.block_until_ready()

with jax.profiler.trace(TRACES_DIR):
    result = gemm2_compiled(inputs, weights)
    result.block_until_ready()

In [12]:
@functools.partial(
    jax.shard_map,
    mesh=mesh,
    in_specs=(P('x', 'y'), P('x', None)),
    out_specs=P('x', None),
    check_vma=False
)
def xla_matmul2(input_shard: jax.Array, weight_shard: jax.Array) -> jax.Array:
    """
    This time, we want to make the computation a little more efficient than
    stacking the two all gathers are the beginning of the kernel
    """
    y_idx = jax.lax.axis_index('y')
    # All gather the weights over x so that each device contains full copy
    w_full = jax.lax.all_gather(weight_shard, 'x', axis=0, tiled=True)
    # Using the y-ring axis to determined which col stripe of weights to compute locally
    w_slice = jax.lax.dynamic_slice(w_full, (y_idx * 1024, 0), (1024, 1024))
    local_out = input_shard @ w_slice
    # All Reduce over the y-ring to accumulate partial results
    out = jax.lax.psum(local_out, 'y')
    return out

In [13]:
a = xla_matmul2(inputs, weights)

In [14]:
allclose(basic_matmul(inputs, weights), xla_matmul2(inputs, weights))

False

In [15]:
@functools.partial(
    jax.shard_map,
    mesh=mesh,
    in_specs=(P('x', 'y'), P('x', None)),
    out_specs=P('x', None),
    check_vma=False
)
def xla_matmul3(input_shard: jax.Array, weight_shard: jax.Array) -> jax.Array:
    """
    Use some higher precision numerics to accomodate accumulation order
    """
    y_idx = jax.lax.axis_index('y')
    w_full = jax.lax.all_gather(weight_shard, 'x', axis=0, tiled=True)
    # This shouldn't hardcode the dim shapes
    w_slice = jax.lax.dynamic_slice(w_full, (y_idx * 1024, 0), (1024, 1024))
    # This is probably overkill, think it might also incur perf penalty
    #   - Something in the docs about precision highest
    local_out = jax.lax.dot_general(
        input_shard, w_slice,
        dimension_numbers=(((1,), (0,)), ((), ())),
        precision=jax.lax.Precision.HIGHEST,
        preferred_element_type=jnp.float32,
    )
    out = jax.lax.psum(local_out, 'y')
    return out

In [16]:
jnp.allclose(xla_matmul3(inputs, weights), basic_matmul(inputs, weights), rtol=1e-2, atol=1e-2)

Array(True, dtype=bool)

In [ ]:
"""
Let's recall what we've learned so far --

When needed to perform an all gather on the reduction axis of our weights
to remove the sharding over X

Then, we compute local MatMuls (slicing out the appropriate data) between
the shard local inputs and the full weights
- Recall, we have the full weights after AG, so we need to slice out the
  appropriate chunks of W for our computation

These MatMuls are accumulators -> They finally need to be all reduced
over Y. The desired out sharding is ('x', None), so when we do an AR
over the Y axis, we are sharing the partial results
"""

"""
Let's sketch out the algorithm we care about --

We have 2 arrays distributed over 4 devices
  - 1/4 of inputs on each device
  - 1/2 of weights of each device

We want to efficiently compute this distributed matmul over the devices

We know that the contracting dims are sharded differently

So there will need to be some comms to unshard so that we have the
whole array in the right place. HOWEVER, we may also be able to get
away with ppermute to simple pass _results_ after compute is finished

Here are the kernels we may want to try
  - Simple matmul with lax collectives inserted in the right spots
  - MatMul with handrolled collectives (still AG to start, then AR)
  - ppermute
    - Issue async DMA
    - Run local compute; stash in accumulator
    - 
    - How much latency can we hide here?
      - If the DMAs are fast/slow?
      - How do we _reason_ about these tradeoffs

Start with 2x2 case; don't worry too much about abstracting things out
  - Then extend to larger configurations + abstractions
  - How do we think about the work we're doing?
  - Where are the opportunities to show different edge cases?
  - Where do our assumptions break down?
  - emit_pipeline
  - kernel schedule?
  - What happens when we run this on Trillium?
    - What changes?
"""

In [ ]:
"""
Here's what happens:
- We have 2 arrays in HBM
- Need to all_gather weights over x, now each device has fully copy of x
- Then do local compute
- All Reduce the local compute to get the correct results
"""

#NOTE: THIS IS THE CASE WHERE WE SIMPLY REPLACE jax.dot/jax.matmul/x @ y in xla_matmul3
def simple_matmul(x_ref, y_ref, o_ref, scratch_ref, *, n_steps):
  # Zero scratch buffer
  @pl.when(pl.program_id(2) == 0)
  def _init_scratch():
    scratch_ref[...] = jnp.zeros_like(scratch_ref)

  # Compute dot
  scratch_ref[...] += jnp.dot(
    x_ref[...],
    y_ref[...],
    preferred_element_type=jnp.float32
  )

  # Flush to HBM
  @pl.when(pl.program_id(2) == n_steps - 1)
  def _flush_scratch():
    o_ref[...] = scratch_ref[...].astype(o_ref.dtype)


def make_matmul(
  x: jax.Array,
  y: jax.Array,
  *,
  bm: int = 128,
  bk: int = 128,
  bn: int = 128,
):
  m, k = x.shape
  _, n = y.shape

  grid_spec = pltpu.PrefetchScalarGridSpec(
    num_scalar_prefetch=0,
    grid=(m//bm, n//bn, k//bk),
    in_specs=[
      pl.BlockSpec((bm, bk), lambda i,j,k: (i, k)),
      pl.BlockSpec((bk, bn), lambda i,j,k: (k, j))
    ],
    out_specs=pl.BlockSpec((bm, bn), lambda i, j, k: (i, j)),
    scratch_shapes=[pltpu.VMEM((bm, bn), jnp.float32)]
  )

  return pl.pallas_call(
    functools.partial(simple_matmul, n_steps=k//bk),
    grid_spec=grid_spec,
    # Made this float32 to appease the numerics gods
    out_shape=jax.ShapeDtypeStruct((m, n), dtype=jnp.float32),
    interpret=True
  )(x, y)


def distributed_gemm_kernel1(inputs, weights):
  y_idx = jax.lax.axis_index('y')
  # AG
  w_full = jax.lax.all_gather(weights, 'x', axis=0, tiled=True)
  jax.debug.print('w_full: {}', w_full.shape)
  # Slice out local arrays
  # TODO: again, fix these so they're not tied to the specific shapes
  w_slice = jax.lax.dynamic_slice(w_full, (y_idx * 1024, 0), (1024, 1024))
  jax.debug.print('w_slice: {}', w_slice.shape)
  jax.debug.print('input_shape: {}', inputs.shape)
  # We'll take the default tile sizes for now
  local_out = make_matmul(inputs, w_slice)
  jax.debug.print('local_out: {}', local_out.shape)
  return jax.lax.psum(local_out, 'y')

In [59]:
dgk1 = jax.shard_map(
    distributed_gemm_kernel1,
    mesh=mesh,
    in_specs=(P('x', 'y'), P('x', None)),
    out_specs=P('x', None),
    check_vma=False
)(inputs, weights)

w_full: (Array(2048, dtype=int32), Array(1024, dtype=int32))
w_full: (Array(2048, dtype=int32), Array(1024, dtype=int32))
w_full: (Array(2048, dtype=int32), Array(1024, dtype=int32))
w_full: (Array(2048, dtype=int32), Array(1024, dtype=int32))
w_slice: (Array(1024, dtype=int32), Array(1024, dtype=int32))
w_slice: (Array(1024, dtype=int32), Array(1024, dtype=int32))
w_slice: (Array(1024, dtype=int32), Array(1024, dtype=int32))
w_slice: (Array(1024, dtype=int32), Array(1024, dtype=int32))
input_shape: (Array(1024, dtype=int32), Array(1024, dtype=int32))
input_shape: (Array(1024, dtype=int32), Array(1024, dtype=int32))
input_shape: (Array(1024, dtype=int32), Array(1024, dtype=int32))
input_shape: (Array(1024, dtype=int32), Array(1024, dtype=int32))
local_out: (Array(1024, dtype=int32), Array(1024, dtype=int32))
local_out: (Array(1024, dtype=int32), Array(1024, dtype=int32))
local_out: (Array(1024, dtype=int32), Array(1024, dtype=int32))
local_out: (Array(1024, dtype=int32), Array(1024, dt

In [61]:
jnp.allclose(dgk1, basic_matmul(inputs, weights), rtol=1e-2, atol=1e-2)

Array(True, dtype=bool)

In [62]:
ref = basic_matmul(inputs, weights)
test = dgk1
diff = jnp.abs(ref - test)

print(f"max diff: {jnp.max(diff)}")
print(f"mean diff: {jnp.mean(diff)}")
print(f"median diff: {jnp.median(diff)}")
print(f"% > 0.1: {100 * jnp.mean(diff > 0.1):.2f}%")

# Location of errors
bad_mask = diff > 0.1
bad_rows = jnp.any(bad_mask, axis=1)
bad_cols = jnp.any(bad_mask, axis=0)
print(f"rows with errors: {jnp.sum(bad_rows)}  {ref.shape[0]}")
print(f"cols with errors: {jnp.sum(bad_cols)} {ref.shape[1]}")

# Worst error location
bad_idx = jnp.argmax(diff)
i, j = bad_idx // ref.shape[1], bad_idx % ref.shape[1]
print(f"worst error at [{i}, {j}]: ref={ref[i,j]}, test={test[i,j]}")

# Check quadrants (shard boundaries)
quadrants = [
    ("top-left", diff[:1024, :512]),
    ("top-right", diff[:1024, 512:]),
    ("bottom-left", diff[1024:, :512]),
    ("bottom-right", diff[1024:, 512:]),
]
for name, q in quadrants:
    mean_val = float(jnp.mean(q))
    pct_bad = float(100 * jnp.mean(q > 0.1))
    print(f"{name}: mean={mean_val:.4f}, % bad={pct_bad:.1f}%")

max diff: 0.5
mean diff: 0.05057927221059799
median diff: 0.031246185302734375
% > 0.1: 15.81%
rows with errors: 2048  2048
cols with errors: 1024 1024
worst error at [875, 791]: ref=128, test=128.5
top-left: mean=0.0507, % bad=15.9%
top-right: mean=0.0506, % bad=15.8%
bottom-left: mean=0.0505, % bad=15.8%
bottom-right: mean=0.0505, % bad=15.7%


In [ ]:
# This may not be a part of the "build it up" story quite yet...
# May need one more kernel to handle things
def matmul_kernel_pipelining(x_ref, y_ref, o_ref):
  # Prologue

  # Need to make the iteration space generic over axis sizes
  # pl.program_id(...) -> The AGs/ARs will run N-1 times

  # Compute
  # Should probably have an inner pipeline schedule for the GEMM
  # I think that this should work?
  # emit_pipeline(...)

  # Epilogue

  pass

# Grid should be N-1 times?
# Or... (Grid - 1, Grid - 1)
# This way we have the initial AllGather
# There is an inner pipeline per iteration
# 



def matmul_kernel_async():
    pass

In [ ]:
# NUMERICS CHECK

# ACCURACY CHECK

In [ ]:
# PERFORMANCE CHECK

In [ ]:
# PROFILING

In [ ]:
# EXTEND THE KERNEL TO BE MORE GENERAL THAN 2x2
# REPEAT CHECKS
# SCALE UP TO 4x4 GRID